# Análise de Resultados do Modelo

Neste notebook vamos:
1. Carregar o modelo treinado
2. Avaliar performance no conjunto de teste
3. Visualizar matriz de confusão
4. Analisar erros comuns
5. Visualizar curvas de aprendizado
6. Testar em imagens específicas

In [1]:
import sys
from pathlib import Path

# Adicionar o diretório raiz ao path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from src.utils.config_loader import config
from src.emotion_recognition.data_loader import EmotionDataLoader

# Configurar estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

%matplotlib inline

/Users/milenemartins/Documents/projetos-pessoais/neuroguide/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. Carregar Dados e Modelo

In [2]:
# Configurações
paths = config.get_paths()
emotion_classes_pt = config['dataset']['classes']

# Mapear para inglês (nomes nas pastas)
emotion_classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# ALTERE AQUI: Coloque o nome do seu modelo treinado
MODEL_NAME = 'emotion_cnn_XXXXXX_best.h5'  # Substitua XXXXXX pelo timestamp

model_path = paths['models_dir'] / MODEL_NAME

if not model_path.exists():
    print(f"❌ Modelo não encontrado: {model_path}")
    print("\nModelos disponíveis:")
    for m in sorted(paths['models_dir'].glob('*.h5')):
        print(f"  - {m.name}")
    print("\n⚠️ Atualize MODEL_NAME acima com um dos modelos listados")
else:
    print(f"✅ Carregando modelo: {model_path.name}")
    model = load_model(model_path)
    print("✓ Modelo carregado com sucesso!")

❌ Modelo não encontrado: /Users/milenemartins/Documents/projetos-pessoais/neuroguide/models/emotion_cnn_XXXXXX_best.h5

Modelos disponíveis:

⚠️ Atualize MODEL_NAME acima com um dos modelos listados


In [3]:
# Carregar dados de teste usando generator
print("Carregando dados de teste...")

data_loader = EmotionDataLoader(
    data_path=paths['data_dir'],
    img_size=tuple(config['dataset']['img_size']),
    num_classes=config['dataset']['num_classes'],
    color_mode=config['dataset']['color_mode']
)

# Criar apenas test generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    str(paths['data_dir'] / 'test'),
    target_size=tuple(config['dataset']['img_size']),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False  # Importante para análise
)

print(f"\n✓ Dados de teste carregados:")
print(f"  Total de amostras: {test_generator.samples}")
print(f"  Classes: {list(test_generator.class_indices.keys())}")

Carregando dados de teste...
Found 7178 images belonging to 7 classes.

✓ Dados de teste carregados:
  Total de amostras: 7178
  Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


## 2. Avaliar Performance

In [4]:
# Fazer predições
print("Fazendo predições no conjunto de teste...")
test_generator.reset()

y_pred_proba = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = test_generator.classes

# Métricas gerais
test_generator.reset()
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator), verbose=0)

print(f"\n{'='*50}")
print(f"RESULTADOS NO CONJUNTO DE TESTE")
print(f"{'='*50}")
print(f"Loss: {test_loss:.4f}")
print(f"Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"{'='*50}")

Fazendo predições no conjunto de teste...


NameError: name 'model' is not defined

In [ ]:
# Relatório de classificação detalhado
print("\nRelatório de Classificação por Emoção:\n")
print(classification_report(y_true, y_pred, target_names=emotion_classes))

## 3. Matriz de Confusão

In [ ]:
# Calcular matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plotar
plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=emotion_classes,
    yticklabels=emotion_classes,
    cbar_kws={'label': 'Quantidade'}
)
plt.title('Matriz de Confusão - Reconhecimento de Emoções', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Emoção Prevista', fontsize=12, fontweight='bold')
plt.ylabel('Emoção Real', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Matriz normalizada (percentual)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm_normalized,
    annot=True,
    fmt='.2%',
    cmap='YlOrRd',
    xticklabels=emotion_classes,
    yticklabels=emotion_classes,
    cbar_kws={'label': 'Proporção'}
)
plt.title('Matriz de Confusão Normalizada (%)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Emoção Prevista', fontsize=12, fontweight='bold')
plt.ylabel('Emoção Real', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 4. Análise de Erros

In [ ]:
# Identificar confusões mais comuns
confusion_pairs = []

for i in range(len(emotion_classes)):
    for j in range(len(emotion_classes)):
        if i != j and cm[i, j] > 0:
            confusion_pairs.append({
                'Real': emotion_classes[i],
                'Previsto': emotion_classes[j],
                'Quantidade': cm[i, j],
                'Percentual': cm_normalized[i, j] * 100
            })

confusion_df = pd.DataFrame(confusion_pairs).sort_values('Quantidade', ascending=False)

print("\nTop 10 Confusões Mais Comuns:")
print("="*60)
display(confusion_df.head(10))

In [ ]:
# Visualizar exemplos de erros
def plot_misclassified_examples(n_examples=10):
    """Plota exemplos de imagens classificadas incorretamente"""
    
    # Encontrar índices de predições erradas
    wrong_indices = np.where(y_pred != y_true)[0]
    
    if len(wrong_indices) == 0:
        print("Nenhum erro encontrado!")
        return
    
    # Selecionar aleatoriamente alguns exemplos
    sample_indices = np.random.choice(wrong_indices, size=min(n_examples, len(wrong_indices)), replace=False)
    
    # Obter imagens do generator
    test_generator.reset()
    images = []
    labels = []
    
    for i in range(len(test_generator)):
        batch_images, batch_labels = test_generator[i]
        images.extend(batch_images)
        labels.extend(batch_labels)
        if len(images) >= test_generator.samples:
            break
    
    # Plotar
    n_cols = 5
    n_rows = (n_examples + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4*n_rows))
    axes = axes.flatten() if n_examples > 1 else [axes]
    
    fig.suptitle('Exemplos de Classificações Incorretas', fontsize=16, fontweight='bold')
    
    for idx, ax in zip(sample_indices, axes):
        img = images[idx].squeeze()
        true_emotion = emotion_classes[y_true[idx]]
        pred_emotion = emotion_classes[y_pred[idx]]
        confidence = y_pred_proba[idx][y_pred[idx]] * 100
        
        ax.imshow(img, cmap='gray')
        ax.set_title(
            f"Real: {true_emotion}\nPrevisto: {pred_emotion}\nConf: {confidence:.1f}%",
            fontsize=10,
            color='red'
        )
        ax.axis('off')
    
    # Esconder eixos extras
    for ax in axes[len(sample_indices):]:
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

plot_misclassified_examples(10)

## 5. Curvas de Aprendizado

In [ ]:
# Carregar histórico (se disponível)
history_file = model_path.parent / model_path.name.replace('_best.h5', '_history.npz')

if history_file.exists():
    history_data = np.load(history_file)
    
    train_loss = history_data['train_loss']
    train_acc = history_data['train_accuracy']
    val_loss = history_data['val_loss']
    val_acc = history_data['val_accuracy']
    
    epochs = range(1, len(train_loss) + 1)
    
    # Plotar Loss e Accuracy
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))
    
    # Loss
    ax1.plot(epochs, train_loss, 'b-', label='Training Loss', linewidth=2)
    ax1.plot(epochs, val_loss, 'r-', label='Validation Loss', linewidth=2)
    ax1.set_title('Curva de Loss', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Marcar melhor epoch
    best_epoch = np.argmin(val_loss) + 1
    ax1.axvline(best_epoch, color='green', linestyle='--', alpha=0.7, label=f'Best Epoch ({best_epoch})')
    
    # Accuracy
    ax2.plot(epochs, train_acc, 'b-', label='Training Accuracy', linewidth=2)
    ax2.plot(epochs, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
    ax2.set_title('Curva de Acurácia', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy', fontsize=12)
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    # Marcar melhor epoch
    best_epoch_acc = np.argmax(val_acc) + 1
    ax2.axvline(best_epoch_acc, color='green', linestyle='--', alpha=0.7, label=f'Best Epoch ({best_epoch_acc})')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas
    print(f"\n📊 Estatísticas do Treinamento:")
    print(f"  Melhor Validation Accuracy: {max(val_acc):.4f} (Epoch {np.argmax(val_acc) + 1})")
    print(f"  Melhor Validation Loss: {min(val_loss):.4f} (Epoch {np.argmin(val_loss) + 1})")
    print(f"  Total de Epochs: {len(epochs)}")
    print(f"  Training Accuracy final: {train_acc[-1]:.4f}")
    print(f"  Validation Accuracy final: {val_acc[-1]:.4f}")
else:
    print(f"❌ Arquivo de histórico não encontrado: {history_file}")

## 6. Análise por Emoção

In [ ]:
# Métricas por emoção
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)

metrics_df = pd.DataFrame({
    'Emoção': emotion_classes,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print("\nMétricas por Emoção:")
display(metrics_df.sort_values('F1-Score', ascending=False))

# Plotar
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(emotion_classes))
width = 0.25

ax.bar(x - width, precision, width, label='Precision', alpha=0.8)
ax.bar(x, recall, width, label='Recall', alpha=0.8)
ax.bar(x + width, f1, width, label='F1-Score', alpha=0.8)

ax.set_xlabel('Emoção', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Métricas por Emoção', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(emotion_classes, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1.1])

plt.tight_layout()
plt.show()